In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from top2vec import Top2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path_corpus_tut = Path(r'..\..\data\corpora\clean_tutby_126784.csv')
path_model_top2vec = Path(r'..\..\data\model\t2v\top2vec_reduced_100_clean_126784.joblibfile')
reduced = True

In [3]:
model = Top2Vec.load(path_model_top2vec)

n_topics = model.get_num_topics(reduced=reduced)
print(n_topics)

100


In [4]:
data = pd.read_csv(path_corpus_tut)

topic = model.doc_top_reduced
words = model.topic_words_reduced[topic]

data['topic'] = topic
data['topic_words'] = words[:,:5].tolist()

data = data[data['document'].notna()]

print(data.shape)
display(data.head(5))

(126773, 8)


,url,label,header,date,document,tags,topic,topic_words
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол'],1,"[бате, борисовчанин, городея, еврокубок, еврок..."
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']",40,"[подъезд, жилец, жэс, заделывать, жэу]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства'],32,"[художник, артобъект, скульптура, инсталляция,..."
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']",1,"[бате, борисовчанин, городея, еврокубок, еврок..."
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево...",84,"[порошенко, батькивщина, саакашвили, зеленскии..."


In [5]:
%%time

topic_docs = data.groupby('topic').agg({'document': ' '.join})

vectorizer_tf = TfidfVectorizer(norm='l1', use_idf=False)
tf = vectorizer_tf.fit_transform(topic_docs['document'])

vectorizer_idf = TfidfVectorizer(norm='l1', use_idf=True)
vectorizer_idf.fit_transform(data['document'])
idf = vectorizer_idf.idf_

assert vectorizer_tf.get_feature_names() == vectorizer_idf.get_feature_names()
vocabulary = vectorizer_idf.get_feature_names()

tf_idf = np.multiply(tf.todense(), idf ** 2)

top_k = 5
index = tf_idf.argsort(axis=1)[:, -top_k:][:,::-1]

top_words = np.array(vocabulary)[index]

data['topic_words_tf_idf'] = top_words[data['topic']].tolist()


print(tf.shape)
print(idf.shape)
print(tf_idf.shape)
print(index.shape)
print(top_words.shape)
data.head(10)

(100, 391125)
(391125,)
(100, 391125)
(100, 5)
(100, 5)
Wall time: 1min 10s


,url,label,header,date,document,tags,topic,topic_words,topic_words_tf_idf
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол'],1,"[бате, борисовчанин, городея, еврокубок, еврок...","[бате, матч, динамо, команда, шахтер]"
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']",40,"[подъезд, жилец, жэс, заделывать, жэу]","[жилец, подъезд, читатель, дом, ремонт]"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства'],32,"[художник, артобъект, скульптура, инсталляция,...","[художник, музей, скульптура, картина, искусство]"
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']",1,"[бате, борисовчанин, городея, еврокубок, еврок...","[бате, матч, динамо, команда, шахтер]"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево...",84,"[порошенко, батькивщина, саакашвили, зеленскии...","[саакашвили, зеленский, порошенко, украина, на..."
5,https://news.tut.by/550311.html,Футбол,"""Славия"" выбыла в 1/16 Кубка Беларуси, проигра...",2017-07-06T22:48:00+03:00,аутсайдер высокий лига мозырский славия выбыва...,"['футбол', 'Чемпионат Беларуси по футболу']",1,"[бате, борисовчанин, городея, еврокубок, еврок...","[бате, матч, динамо, команда, шахтер]"
6,https://news.tut.by/550313.html,Теннис,Александр Бурый не вышел во 2-й раунд парного ...,2017-07-06T23:05:00+03:00,белорусский теннисист александр бурый выходить...,['теннис'],53,"[ракетка, соболенко, саснович, арина, wta]","[wta, арина, соболенко, азаренко, турнир]"
7,https://news.tut.by/550314.html,В мире,В ходе акции протеста в Гамбурге произошли бес...,2017-07-06T23:13:00+03:00,полиция применять водомет против агрессивный н...,['акции протеста'],46,"[ранить, теракт, полиция, полицеискии, ранение]","[полиция, теракт, полицейский, взрыв, нападение]"
8,https://news.tut.by/550315.html,Общество,"""Гродно моего детства"": расскажите свою истори...",2017-07-14T15:12:00+03:00,помнить ездить смотреть город будущее самолетк...,"['конкурс', 'история Беларуси']",11,"[фотозона, мастеркласс, мастерклассы, фотозон,...","[фестиваль, праздник, концерт, мастеркласс, вы..."
9,https://news.tut.by/550316.html,GO,ЗОЖ Владимира Япринцева: Я не считаю это заряд...,2017-07-12T10:13:00+03:00,большой спорт большой бизнес слабый дух многок...,"['здоровый образ жизни', 'курение', 'Личный ЗО...",25,"[тренажерка, тренировка, тренироваться, похуде...","[тренировка, упражнение, спорт, занятие, post]"


In [10]:
# %%time

# vectorizer_tf = TfidfVectorizer(norm='l1', use_idf=False)
# tf_corpus = vectorizer_tf.fit_transform([' '.join(topic_docs['document'])])
# tf_corpus = tf_corpus.toarray()

# tf_ = tf.toarray()
# tf_[tf_ < 1e-3] = 0

# scores = tf_ / tf_corpus


# index = scores.argsort(axis=1)[:, -top_k:][:,::-1]
# top_words_scores = np.array(vocabulary)[index]
# data['topic_words_scores'] = top_words_scores[data['topic']].tolist()

# data.head(5)

In [7]:
pd.DataFrame(zip(model.topic_words_reduced[:,:5], top_words)).head(50)

,0,1
0,"[дтп, пдд, водитель, гаи, встречок]","[водитель, дтп, авария, гаи, травма]"
1,"[бате, борисовчанин, городея, еврокубок, еврок...","[бате, матч, динамо, команда, шахтер]"
2,"[хоккеист, шаиба, кхл, буллит, овертаим]","[динамо, матч, шайба, кхл, хоккеист]"
3,"[ивс, штатское, омон, окрестин, пикетирование]","[аккумуляторный, тихановский, задерживать, зав..."
4,"[спортсмен, олимпиада, олимпиискии, атлет, мед...","[олимпийский, спортсмен, олимпиада, медаль, сп..."
5,"[самооценка, чувство, бессознательно, стыд, секс]","[женщина, секс, ребенок, психолог, жизнь]"
6,"[бомбардировщик, многоцелевои, надводныи, свер...","[ракета, истребитель, самолет, корабль, ядерный]"
7,"[концерт, танцпол, лсп, баста, бастыи]","[концерт, билет, розыгрыш, песня, клип]"
8,"[санузел, гостиная, кухнягостиная, спальня, пр...","[realtby, квартира, дом, квадратный, кухня]"
9,"[избивать, милиционер, хулиганство, нож, ударять]","[милиция, мужчина, суд, обвиняемый, избивать]"
